In [1]:
import ipyparallel as ipp

from ipyparallel import Cluster
c = await Cluster(engines="mpi", profile="myprofile").start_and_connect(n=1, activate=True)
c.ids

Starting 1 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


  0%|          | 0/1 [00:00<?, ?engine/s]

[0]

In [2]:
%%px
from ngsolve import *
from ngsolve.webgui import Draw
import ngsolve as ngs
import NgsAMG as amg

### H1

It works, I guess.

In [3]:
%%px
from usrMtgStuff import setupH1Square, testAndSolve

V, a, f, u = setupH1Square(maxh=0.01, nref=0)

c = Preconditioner(a, "NgsAMG.h1_scal", ngs_amg_crs_alg="spw")
a.Assemble()
f.Assemble()

testAndSolve(a.mat, c, u.vec, f.vec)

[stdout:0] Set up AMG-levels...
 map maps 11788 -> 1467, fac 0.124449
 map maps 1467 -> 205, fac 0.139741
 map maps 205 -> 31, fac 0.15122
Done setting up AMG-levels!

 ---------- AMG Summary ---------- 
Vertex complexity: 1.14447
Operator complexity: 1.32634
Vertex complexity components: 1 0.124449 0.0173906 0.00262979 
Operator complexity components: 1 0.260201 0.0595398 0.0065965 
# vertices 11788 1467 205 31 
# edges: 34961 4294 577 77 
# procs: 1 1 1 1 
NZEs:81710 21261 4865 539 
 ---------- AMG Summary End ---------- 

CG iteration 1, residual = 0.15548822132496148     
CG iteration 2, residual = 0.02964104842767322     
CG iteration 3, residual = 0.007957606249164022     
CG iteration 4, residual = 0.0018523499748401757     
CG iteration 5, residual = 0.0004201448396123619     
CG iteration 6, residual = 9.78991811471367e-05     
CG iteration 7, residual = 2.218593203097147e-05     
CG iteration 8, residual = 5.110384918563826e-06     
CG iteration 9, residual = 1.23056395551637

%px:   0%|          | 0/1 [00:00<?, ?tasks/s]

Out[0:2]: 2.6351741689612016

In [4]:
%%px
Draw(u);

[output:0]

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

### Elasticity

In [6]:
%%px
from usrMtgStuff import setupElastBeam, testAndSolve

V, a, f, u = setupElastBeam(N=10, dim=3)

c = Preconditioner(a, f"NgsAMG.elast_{V.mesh.dim}d", ngs_amg_crs_alg="mis")
a.Assemble()
f.Assemble()

testAndSolve(a.mat, c, u.vec, f.vec)

[stdout:0] Set up AMG-levels...
 map maps 12221 -> 828, fac 0.0677522
 map maps 828 -> 67, fac 0.0809179
 map maps 67 -> 7, fac 0.104478
Done setting up AMG-levels!

 ---------- AMG Summary ---------- 
Vertex complexity: 1.07381
Operator complexity: 1.58873
Vertex complexity components: 1 0.0677522 0.00548237 0.000572785 
Operator complexity components: 1 0.557661 0.0300253 0.00104204 
# vertices 12221 828 67 7 
# edges: 76420 4592 248 7 
# procs: 1 1 1 1 
NZEs:165061 23012 1239 43 
 ---------- AMG Summary End ---------- 

CG iteration 1, residual = 0.0004162600363685324     
CG iteration 2, residual = 0.00023155095129443994     
CG iteration 3, residual = 9.063800529889062e-05     
CG iteration 4, residual = 4.271695228312529e-05     
CG iteration 5, residual = 1.812705998371707e-05     
CG iteration 6, residual = 7.115183374698533e-06     
CG iteration 7, residual = 3.013402533595782e-06     
CG iteration 8, residual = 1.355083584424406e-06     
CG iteration 9, residual = 6.351378275

Out[0:5]: 7.277048252624605

In [9]:
%%px
Draw(u, deformation=True)

[output:0]

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

Out[0:7]: BaseWebGuiScene

No silver bullet - similar limitations as PETSc, BoomerAMG, etc

In [8]:
%%px
from usrMtgStuff import setupElastBeam, testAndSolve

V, a, f, u = setupElastBeam(N=10, aRatio=1000, dim=2)

c = Preconditioner(a, f"NgsAMG.elast_{V.mesh.dim}d", ngs_amg_crs_alg="spw", ngs_amg_sm_type="bgs", ngs_amg_sp_improve_its=2, ngs_amg_sp_omega=0.8)
# c = Preconditioner(a, f"NgsAMG.elast_{V.mesh.dim}d", ngs_amg_crs_alg="mis", ngs_amg_on_dofs="select", ngs_amg_subset= "nodalp2", ngs_amg_sm_type="bgs")
# c = Preconditioner(a, f"NgsAMG.elast_{V.mesh.dim}d", ngs_amg_crs_alg="mis", ngs_amg_lo = False, ngs_amg_dof_ordering="p2Emb", ngs_amg_smooth_after_emb=False, ngs_amg_sp_improve_its=0, ngs_amg_sm_type="bgs")

a.Assemble()
f.Assemble()

testAndSolve(a.mat, c, u, f.vec)

[stdout:0] Set up AMG-levels...
 map maps 110011 -> 13750, fac 0.124988
 map maps 13750 -> 1727, fac 0.1256
 map maps 1727 -> 220, fac 0.127389
 map maps 220 -> 28, fac 0.127273
Done setting up AMG-levels!

 ---------- AMG Summary ---------- 
Vertex complexity: 1.14294
Operator complexity: 1.45023
Vertex complexity components: 1 0.124988 0.0156984 0.0019998 0.00025452 
Operator complexity components: 1 0.358098 0.078571 0.0123344 0.00122666 
# vertices 110011 13750 1727 220 28 
# edges: 310010 38729 4846 599 60 
# procs: 1 1 1 1 1 
NZEs:730031 116188 25493 4002 398 
 ---------- AMG Summary End ---------- 



%px:   0%|          | 0/1 [00:00<?, ?tasks/s]

[0:execute]
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
Cell In[6], line 12
      9 a.Assemble()
     10 f.Assemble()
---> 12 testAndSolve(a.mat, c, u, f.vec)

File ~/ngsolve/usrMtg/presentations/usrMtg2025/usrMtgStuff.py:117, in testAndSolve(A, c, u, f, doTest, tol)
    114 MPI.COMM_WORLD.Barrier()
    115 t.Start()
--> 117 u.data = sol * f
    119 MPI.COMM_WORLD.Barrier()
    120 t.Stop()

AttributeError: property of 'GridFunction' object has no setter


AlreadyDisplayedError: 1 errors

In [ ]:
%%px
from usrMtgStuff import setupElastBeam, testAndSolve

V, a, f, u = setupElastBeam(N=10, elStretch=5, dim=3, order=2, nodalP2=True)

# c = Preconditioner(a, f"NgsAMG.elast_{V.mesh.dim}d", ngs_amg_crs_alg="mis")
# c = Preconditioner(a, f"NgsAMG.elast_{V.mesh.dim}d", ngs_amg_crs_alg="mis", ngs_amg_on_dofs="select", ngs_amg_subset= "nodalp2", ngs_amg_sm_type="bgs")
c = Preconditioner(a, f"NgsAMG.elast_{V.mesh.dim}d", ngs_amg_crs_alg="mis", ngs_amg_lo = False, ngs_amg_dof_ordering="p2Emb", ngs_amg_smooth_after_emb=False, ngs_amg_sp_improve_its=0, ngs_amg_sm_type="bgs")

a.Assemble()
f.Assemble()

testAndSolve(a.mat, c, u.vec, f.vec)

print(f"\n\n\ty-defo = {u(1.0,0.0)[1]} !")

### HDiv

Coarsens dual graph, maintains "loops" (i.e. vertices/edges in 2d/3d), robust in $\left<\nabla\cdot,\nabla\cdot\right>$ penalty parameter

In [ ]:
%%px
def StokesHDGDiscretization(mesh, order, inlet, wall, outlet, hodivfree, proj_jumps, div_div_pen, with_pressure, V, Vhat, nu, diri, elint = ('', '', '', True, True, None, True, None, None, 1, 'wall|inlet', False)):
    V1 = HDiv(mesh, order, diri, hodivfree, False, **('order', 'dirichlet', 'hodivfree', 'RT'))
    V2 = TangentialFacetFESpace(mesh, order, diri, proj_jumps, **('order', 'dirichlet', 'highest_order_dc'))
    V = V1 * V2
    Q = L2(mesh, 0 if hodivfree else order - 1, **('order',))
    (u, uhat) = ()
    (v, vhat) = V.TnT()
    (p, q) = Q.TnT()
    n = specialcf.normal(mesh.dim)
    h = specialcf.mesh_size
    
    def tang(vec = None):
        return vec - vec * n * n

    alpha = 4
    dS = dx(True, **('element_boundary',))
    a = BilinearForm(V, elint, **('eliminate_internal',))
    a += nu * InnerProduct(Grad(u), Grad(v)) * dx
    a += nu * InnerProduct(Grad(u) * n, tang(vhat - v)) * dS
    a += nu * InnerProduct(Grad(v) * n, tang(uhat - u)) * dS
    a += (nu * alpha * order * order / h) * InnerProduct(tang(vhat - v), tang(uhat - u)) * dS
    if div_div_pen is not None:
        aPen = BilinearForm(V, elint, **('eliminate_internal',))
        aPen += nu * InnerProduct(Grad(u), Grad(v)) * dx
        aPen += nu * InnerProduct(Grad(u) * n, tang(vhat - v)) * dS
        aPen += nu * InnerProduct(Grad(v) * n, tang(uhat - u)) * dS
        aPen += (nu * alpha * order * order / h) * InnerProduct(tang(vhat - v), tang(uhat - u)) * dS
        aPen += div_div_pen * nu * InnerProduct(div(u), div(v)) * dx
    else:
        aPen = a
    b = BilinearForm(V, Q, **('trialspace', 'testspace'))
    b += -div(u) * q * dx
    return (V, Q, a, b, aPen)

In [ ]:
%%px
from usrMtgStuff import GetValve

dim = 2

valve = GetValve(1, dim, 0.5, 25, 1, 180, 6.4, 7, 5, True, **('N', 'dim', 'R', 'alpha', 'Winlet', 'beta', 'L1', 'L2', 'Linlet', 'closevalve'))
mesh = Mesh(OCCGeometry(valve, dim, **('dim',)).GenerateMesh(0.5, **('maxh',)))
mesh.Curve(3)
diri = 'wall'
outlet = None
uin = None
f_vol = CF((1, 0))

(V, Q, a, b, aPen) = StokesHDGDiscretization(mesh, order=order, diri=diri, nu=nu, div_div_pen=div_div_pen)

amg_cl = NgsAMG.stokes_hdiv_gg_2d if mesh.ngmesh.dim == 2 else NgsAMG.stokes_hdiv_gg_3d

pc_opts = {
    'ngs_amg_max_levels': 40,
    'ngs_amg_max_coarse_size': 1,
    'ngs_amg_clev': 'inv',
    'ngs_amg_log_level': 'extra',
    'ngs_amg_log_level_pc': 'extra',
    'ngs_amg_do_test': True,
    'ngs_amg_mg_cycle': 'V',
    'presVecs': 'P1',
    "ngs_amg_pres_vecs": ["RTZ", "P0", "P1"][0]
}

